In [4]:
!pip install requests-toolbelt

     -------------------------------------- 54.5/54.5 kB 353.9 kB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!python.exe -m pip install --upgrade pip

     ---------------------------------------- 2.1/2.1 MB 782.6 kB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1


In [8]:
#-*- encoding:utf-8 -*-
import requests
import json
from requests_toolbelt import MultipartEncoder

class qywx:
    corpid = 'wwdb706c66c9567107' #步骤1-3中的企业id
    # 此处将步骤1-2中的AgentId 和 Secret分别填入，多个应用逗号隔开
    app_list = [
        (1000003, 'RL8K0UDfGz0O0YqzFeJcp0gCNyl4kGc607FEmN0Yg64'), # QMTTest(需设置企业可信IP)
        (1000002, 'JQ37HXmcASXMNePoXh1KoqCiFG4k4Zq4MxuZ46mK_ow'), # QuantRobot
        ]

    def __init__(self, app_id):  #app_id 按app_list顺序编号
        self.agentid, self.corpsecret = qywx.app_list[app_id]

    # 上传临时文件素材接口，图片也可使用此接口，20M上限
    def post_file(self, filepath, filename):
        response = requests.get("https://qyapi.weixin.qq.com/cgi-bin/gettoken?corpid={corpid}&corpsecret={corpsecret}".format(corpid=qywx.corpid, corpsecret=self.corpsecret))
        data = json.loads(response.text)
        access_token = data['access_token']

        post_file_url = "https://qyapi.weixin.qq.com/cgi-bin/media/upload?access_token={access_token}&type=file".format(access_token=access_token)

        m = MultipartEncoder(
            fields={'file': (filename, open(filepath + filename, 'rb'), 'multipart/form-data')},
        )
        
        r = requests.post(url=post_file_url, data=m, headers={'Content-Type': m.content_type})
        js = json.loads(r.text) 
        print("upload " + js['errmsg'])
        if js['errmsg'] != 'ok':
            return None
        return js['media_id']
        
    # 向应用发送图片接口，_message为上传临时素材后返回的media_id
    def send_img(self, _message, useridlist = ['name1|name2']): 
        useridstr = "|".join(useridlist)
        response = requests.get("https://qyapi.weixin.qq.com/cgi-bin/gettoken?corpid={corpid}&corpsecret={corpsecret}".format(corpid=qywx.corpid, corpsecret=self.corpsecret))
        data = json.loads(response.text)
        access_token = data['access_token']

        json_dict = {
            "touser" : useridstr,
            "msgtype" : "image",
            "agentid" : self.agentid,
            "image" : { 
                "media_id" : _message,
            },
            "safe": 0,
            "enable_id_trans": 1,
            "enable_duplicate_check": 0,
            "duplicate_check_interval": 1800
        }
        json_str = json.dumps(json_dict)
        response_send = requests.post("https://qyapi.weixin.qq.com/cgi-bin/message/send?access_token={access_token}".format(access_token=access_token), data=json_str)
        print("send to " + useridstr + ' ' + json.loads(response_send.text)['errmsg'])
        return json.loads(response_send.text)['errmsg'] == 'ok'

    # 向应用发送文字消息接口，_message为字符串
    def send_text(self, _message, useridlist = ['name1|name2']): 
        useridstr = "|".join(useridlist) # userid 在企业微信-通讯录-成员-账号
        response = requests.get("https://qyapi.weixin.qq.com/cgi-bin/gettoken?corpid={corpid}&corpsecret={corpsecret}".format(corpid=qywx.corpid, corpsecret=self.corpsecret))
        data = json.loads(response.text)
        access_token = data['access_token']
        json_dict = {
            "touser" : useridstr,
            "msgtype" : "text",
            "agentid" : self.agentid,
            "text" : {
                "content" : _message
            },
            "safe": 0,
            "enable_id_trans": 1,
            "enable_duplicate_check": 0,
            "duplicate_check_interval": 1800
        }
        json_str = json.dumps(json_dict)
        response_send = requests.post("https://qyapi.weixin.qq.com/cgi-bin/message/send?access_token={access_token}".format(access_token=access_token), data=json_str)
        print("send to " + useridstr + ' ' + json.loads(response_send.text)['errmsg'])
        return json.loads(response_send.text)['errmsg'] == 'ok'
# 调用示例
if __name__ == '__main__':
    qy = qywx(1)
    
    # 发送文本消息
    qy.send_text('hello world', ['ZhuGaoPing'])
    
    # 发送图片消息, 需先上传至临时素材
#     media_id = qy.post_file('/root/', 'a.jpg')
#     qy.send_img(media_id, ['ZhuGaoPing'])

send to ZhuGaoPing ok
